In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import re
import datetime
from sklearn.linear_model import LinearRegression

os.chdir(r"C:\Users\Vasiliy Poteriaev\Google Диск\analytics\auto_ru\data")
dfs = [f for f in os.listdir() if f.endswith('.pickle')]
dfs

['honda_accord_2016_04_05.pickle',
 'hyundai_i40_2016_04_05.pickle',
 'hyundai_sonata_2016_04_06.pickle',
 'kia_optima_2016_04_05.pickle',
 'mazda_6_2016_04_06.pickle',
 'toyota_camry_2016_04_06.pickle']

In [2]:
# concantinating car files
df = pd.read_pickle(dfs[0])
for file in dfs[1:]:
    df = pd.concat([df, pd.read_pickle(file)])

In [3]:
def get_autocod_index(col_names):
    for c, i in zip(col_names, range(len(col_names))):
        if len(c) > 200:
            autocod = i
    return autocod

cols = list(df.columns)
cols[get_autocod_index(cols)] = 'autocode'
df.columns = cols

# Functions for feature calculations

In [4]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, str(j))
    return text


def findOccurences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def replace_seller_location(df, location_series):
    locations = pd.DataFrame(df[location_series].value_counts()).reset_index()
    locations = locations[locations['Seller_location'] >= 5]
    df.ix[~df.Seller_location.isin(locations['index']),location_series] = 'Москва, прочие адреса'
    
def get_own_time(text):
    if type(text) == str:
        #function return owning time of last owner in years

        if (re.search('год', text) != None):
            match_year = re.search('год', text)
            year = float(text[:match_year.start() - 1])
        else:
            year = 0

        if (re.search('месяц', text) != None):
            match_month = re.search('месяц', text)
            month_end_pos = match_month.start() - 1        

            if (re.search(' и ', text) != None):
                match_and = re.search(' и ', text)
                month_start_pos = match_and.end()
            else:
                month_start_pos = 0

            month = float(text[month_start_pos:month_end_pos])
        else:
            month = 0
        owntime = year + month / 12
    else:
        owntime = text
    return owntime

def end_warranty(end_text):
    if re.search('До',str(end_text)) != None:
        result = pd.to_datetime('01 ' + replace_all(end_text,mon)[3:], format = '%d %m %Y')
    else:
        result = np.nan
    return result

def get_engine_capacity(engine):
    match_capacity = re.search(' л', engine)
    capacity = float(engine[:match_capacity.start()])
    return capacity

def get_engine_hp(engine):
    match_hp = re.search('л.с.', engine)
    match_capacity = re.search(' л', engine)
    hp = int(engine[match_capacity.end():match_hp.start()-1])
    return hp

def get_engine_fuel_type(engine):
    match_hp = re.search('л.с.', engine)
    ft = engine[match_hp.end():]
    return ft

def get_run(run):
    if re.search('км',str(run)) != None:
        r = int(run.replace(' км','').replace(' ',''))
    else:
        r = np.nan
    return r

def get_metallic_color(col):
    res = 0
    if type(col) == str:
        if ('металлик' in col):
            res = 1
    return res

def get_color(col):
    res = np.nan
    if type(col) == str:
        col = re.sub('\xa0', '', col)
        res = col
        if (re.search('металлик', col) != None):
            metallic_match = re.search('металлик', col)
            res = col[:metallic_match.start()-1]
    return res

def uppend_to_list(full_list, list_to_uppend):
    for el in list_to_uppend:
        if el not in full_list:
            full_list.append(el)
    return full_list

## Add date to date format 

In [5]:
mon ={'апреля':4,'марта':3,'февраля':2,'июня':6,'августа':8,'декабря':12,
      'ноября':11,'октября':10,'сентября':9,'января':1,'июля':7,'мая':5}

In [6]:
df['Add date'] = df['Add date'].apply(lambda x: replace_all(x,mon))
df['Add date'] = pd.to_datetime(df['Add date'], format = "%d %m %Y")

## Feature generation

In [7]:
#age of advertisement
df_pr = pd.DataFrame({'Advertising_num_days':(datetime.datetime.now().date()-df['Add date'])/ np.timedelta64(1, 'D')})

#extract car base name from 'Brand'
df_pr['CarName'] = df['Brand'].apply(lambda x: x[:findOccurences(x, " ")[2]])

#ID
df_pr['ID'] = df['ID'].astype(int)

#price
df_pr['Price_rub'] = df['Price_rub'].astype(int)

#if seller is not a company summarize names
df_pr['Seller'] = df['Seller']
df_pr.loc[df_pr.Seller.apply(lambda x: "Частное лицо" in x), 'Seller'] = 'person'

sellers =df_pr.groupby('Seller').size().reset_index()
sellers = sellers[sellers[0] < 5]

df_pr.loc[df_pr.Seller.isin(sellers.Seller), 'Seller'] = 'other'

#seller location
df_pr['Seller_location'] = df['Seller_location']
replace_seller_location(df_pr, 'Seller_location')

#Views
df_pr['Views'] = df['Views'].astype(int)

# car age
df_pr['Car_age'] = df['Год выпуска'].apply(lambda x: 2016 - int(x[:4]))
#df_pr['CarAge_sq'] = df['Год выпуска'].apply(lambda x: (2016 - int(x[:4])) ** 2)
# gear box type
df_pr['Gear_box'] = df['Коробка']
# parsing date
df_pr['Parsing date'] = pd.to_datetime(df['Parsing date'], format = '%Y-%m-%d')

#autocode
df_pr['Autocode_checked'] =  df['autocode']
df_pr.ix[~df_pr.Autocode_checked.isnull(),'Autocode_checked'] = 1
df_pr.Autocode_checked = df_pr.Autocode_checked.fillna(0)

#previous owners number
df_pr['Owners_N'] = df['Владельцы']
df_pr['Owners_N'] = df_pr['Owners_N'].fillna('x')
df_pr['Owners_N'] = df_pr['Owners_N'].apply(lambda x: x[:1])
df_pr.ix[df_pr.Owners_N == 'x','Owners_N'] = 'not mentioned'

#owning time of previous owner
df_pr['Last_owner_time_years'] = df['Владение'].apply(get_own_time)
df_pr['Last_owner_time_years'] = df_pr['Last_owner_time_years'].fillna(df_pr['Last_owner_time_years'].median())

df_pr['Warranty_end'] = df['Гарантия'].apply(end_warranty)
df_pr.ix[~df_pr['Warranty_end'].isnull(), 'Warranty_end'] =(df_pr.ix[~df_pr['Warranty_end'].isnull(), 'Warranty_end'] - \
                                                            df_pr.ix[~df_pr['Warranty_end'].isnull(), 'Parsing date']) \
                                                            / np.timedelta64(1, 'Y')
df_pr['Warranty_end'] = df_pr['Warranty_end'].fillna(0)
df_pr.ix[df_pr['Warranty_end'] < 0,'Warranty_end'] = 0


# get engine characteristics
df_pr['Engine_capacity'] = df['Двигатель'].apply(get_engine_capacity)
df_pr['Engine_hp'] = df['Двигатель'].apply(get_engine_hp)
df_pr['Engine_fuel_type'] = df['Двигатель'].apply(get_engine_fuel_type)

# get body typeэ
df_pr['Body_type'] = df['Кузов']

# swap type
df_pr['Swap_type'] = df['Обмен']
df_pr['Swap_type'] = df_pr['Swap_type'].fillna('Не интересует')

# original car passport
df_pr['Original_car_passport'] = df['ПТС'].replace({'Оригинал':1,'Дубликат':0})
df_pr['Original_car_passport'] = df_pr['Original_car_passport'].fillna(1)

# wheel drive type
df_pr['Wheel_drive_type'] = df['Привод']

# get car run, km
df_pr['run_km'] = df['Пробег'].apply(get_run)

# estimate mean run for car age
mean_run = df_pr.groupby(['Car_age'])['run_km'].mean().reset_index()
mean_run['Car_age2'] = mean_run['Car_age'] ** 2
mean_run = mean_run[mean_run['Car_age'] <= 25]
run_clf = LinearRegression()
run_clf.fit(mean_run[['Car_age','Car_age2']],mean_run['run_km'])

age = df_pr.ix[df_pr['run_km'].isnull(), 'Car_age']
df_pr.ix[df_pr['run_km'].isnull(), 'run_km'] = run_clf.intercept_ + run_clf.coef_[0] * age + run_clf.coef_[1] * age ** 2

# steering wheel side
df_pr['Left_steering'] = df['Руль'].replace({'Левый':1,'Правый':0})
df_pr['Left_steering'] = df_pr['Left_steering'].fillna(1)

# metallic color
df_pr['is_metallic_color'] = df['Цвет'].apply(get_metallic_color)
df_pr['is_metallic_color'] = df_pr['is_metallic_color'].fillna(1)

# color
df_pr['Color'] = df['Цвет'].apply(get_color)
df_pr['Color'] = df_pr['Color'].fillna('Цвет не указан')

In [8]:
# create list with car complectation options
options_list = []
for car_options in df['complectation']:
    uppend_to_list(options_list, car_options)

# add option dummy to dataframe
for option in options_list:
    df_pr['Opt_' + option] = df['complectation'].apply(lambda x: int(option in x))

In [15]:
df_pr.to_csv('C:/Users/Vasiliy Poteriaev/Google Диск/analytics/auto_ru/auto_matrix.csv',sep=';')
df_pr.to_pickle('C:/Users/Vasiliy Poteriaev/Google Диск/analytics/auto_ru/auto_matrix.pickle')

In [105]:
df_pr.head()
#del df_pr

,Advertising_num_days,CarName,ID,Price_rub,Seller,Seller_location,Views,Car_age,Gear_box,Parsing date,Autocode_checked,Owners_N,Last_owner_time_years,Warranty_end,Engine_capacity,Engine_hp,Engine_fuel_type,Body_type,Swap_type,Original_car_passport,Wheel_drive_type,run_km,Left_steering,is_metallic_color,Color,opt_Обивка салона: кожа,opt_Цвет салона: темный,opt_Тонированные стекла,Opt_Обивка салона: кожа,Opt_Цвет салона: темный,Opt_Тонированные стекла,Opt_Ксеноновые фары,"Opt_Легкосплавные диски: 17""",Opt_Штатная,Opt_Бортовой компьютер,Opt_Датчик дождя,Opt_Датчик света,Opt_Круиз-контроль,Opt_Обогрев зеркал,Opt_Омыватель фар,Opt_Усилитель руля,Opt_Центральный замок,Opt_Подогрев руля,Opt_Обогрев сидений,Opt_Количество мест: 4,Opt_Климат: климат-контроль 2-зонный,Opt_Электропривод зеркал,Opt_Стеклоподъемники: электро все,Opt_Сиденье водителя: с памятью положения,Opt_Сиденье пассажира: ручная,Opt_Регулировка руля: в двух плоскостях,Opt_Антиблокировочная система (ABS),Opt_Антипробуксовочная система,Opt_Система курсовой устойчивости,Opt_Парктроник,Opt_Подушки безопасности: передние и боковые,Opt_CD,Opt_Люк на крыше,"Opt_Легкосплавные диски: 18""",Opt_Дистанционный запуск двигателя,Opt_Навигационная система,Opt_Сиденье водителя: электро,Opt_Сиденье пассажира: электро,Opt_Подушки безопасности: водителя и пассажира,Opt_Обратная связь,Opt_Климат: климат-контроль 3-зонный,Opt_TV,Opt_Сиденье водителя: ручная,Opt_Регулировка руля: в одной плоскости,Opt_Обивка салона: комбинированный,Opt_Обивка салона: ткань,Opt_Цвет салона: светлый,Opt_Стеклоподъемники: ручные,Opt_Иммобилайзер,Opt_,Opt_Количество мест: 5,"Opt_Легкосплавные диски: 16""",Opt_Сиденье пассажира: ручная по высоте,Opt_Климат: климат-контроль 1-зонный,"Opt_Легкосплавные диски: 15""",Opt_Регулировка руля: электро,Opt_Стеклоподъемники: электро передние,Opt_Сиденье пассажира: с памятью положения,Opt_Обивка салона: велюр,Opt_Сиденье водителя: ручная по высоте,Opt_Количество мест: 2,Opt_Климат: кондиционер,"Opt_Легкосплавные диски: 12""",Opt_Газобаллонное оборудование,Opt_Количество мест: 6,Opt_Кассетник,Opt_Подушки безопасности: водителя,"Opt_Легкосплавные диски: 20""","Opt_Легкосплавные диски: 13""","Opt_Легкосплавные диски: 19""","Opt_Легкосплавные диски: 14""",Opt_Количество мест: 7
0,9,Kia Optima III,1034370610,920000,person,"Москва, прочие адреса",21712,4,Автоматическая,2016-04-13,0,2,1.583333,0.000000,2.4,180,бензин,Седан,Не интересует,1,Передний,61000.000000,1,1,Черный,True,True,True,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,11,Kia Optima IV,1034188606,1589900,АвтоГЕРМЕС KIA Ярославское ш.,"Москва, ул. Красная Сосна, д.3 (поворот с Ярос...",10720,0,Автоматическая,2016-04-13,0,not mentioned,1.583333,0.000000,2.4,185,бензин,Седан,Не интересует,1,Передний,4767.424532,1,0,Cеребристый,False,False,False,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,Kia Optima III,1035050293,1000000,person,"Москва, прочие адреса",136136,3,Автоматическая,2016-04-13,0,3,1.583333,0.000000,2.4,180,бензин,Седан,Не интересует,1,Передний,23000.000000,1,0,Цвет не указан,True,True,True,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,Kia Optima III,1035014583,931400,"ТЦ ""Кунцево Лимитед""","Москва, ул.Горбунова, д.14 ТЦ ""Кунцево"", отдел...",6363,3,Автоматическая,2016-04-13,0,1,1.583333,0.000000,2.4,180,бензин,Седан,Не интересует,1,Передний,61803.000000,1,0,Серый,False,True,True,0,1,1,0,0,0,1,1,1,1,1,1,0,1,0,1,0,0,1,1,0,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,Kia Optima III,1034952091,1200000,person,"Москва, прочие адреса",17457,2,Автоматическая,2016-04-13,0,1,1.583333,2.718742,2.4,180,бензин,Седан,Не интересует,1,Передний,39000.000000,1,0,Черный,False,False,False,0,0

In [22]:
df.head(3)

,Add date,Brand,ID,Parsing date,Price_rub,Seller,Seller_location,URL,VIN,Views,comment,complectation,autocode,Владельцы,Владение,Гарантия,Год выпуска,Двигатель,Коробка,Кузов,Обмен,ПТС,Привод,Пробег,Руль,Состояние,Статус,Таможня,Цвет
0,2016-04-04,Kia Optima III в Истре,1034370610,2016-04-13,920000,Частное лицо (Валерий),"Истра, Красновидово,",https://auto.ru/cars/used/sale/1034370610-4dbd9/,NaN,21712,,"[Обивка салона: кожа, Цвет салона: темный, Тон...",NaN,2 владельца,NaN,NaN,2012,2.4 л180 л.с.бензин,Автоматическая,Седан,Не интересует,Оригинал,Передний,61 000 км,Левый,Не требует ремонта,NaN,Растаможен,Черный металлик
1,2016-04-02,Kia Optima IV в Москве,1034188606,2016-04-13,1589900,АвтоГЕРМЕС KIA Ярославское ш.,"Москва, ул. Красная Сосна, д.3 (поворот с Ярос...",https://auto.ru/cars/new/sale/1034188606-6b5f96/,XWEGW413*G0****24,10720,комплект:D4S4L6A1FHH380Успейте купить Kia Opti...,"[Люк на крыше, Ксеноновые фары, Легкосплавные ...",NaN,NaN,NaN,NaN,"2016, новый",2.4 л185 л.с.бензин,Автоматическая,Седан,NaN,NaN,Передний,NaN,Левый,NaN,NaN,Растаможен,Cеребристый
2,2016-04-13,Kia Optima III в Москве,1035050293,2016-04-13,1000000,Частное лицо (id16057911),"Москва, м.бунинская аллея",https://auto.ru/cars/used/sale/1035050293-382a2/,NaN,136136,,"[Обивка салона: кожа, Цвет салона: темный, Люк...",NaN,3 владельца,NaN,NaN,2013,2.4 л180 л.с.бензин,Автоматическая,Седан,NaN,Оригинал,Передний,23 000 км,Левый,Не требует ремонта,NaN,Растаможен,Черный


In [10]:
df[(df.ID == '1033486985')]

,Add date,Brand,ID,Parsing date,Price_rub,Seller,Seller_location,URL,VIN,Views,comment,complectation,autocode,Владельцы,Владение,Гарантия,Год выпуска,Двигатель,Коробка,Кузов,Обмен,ПТС,Привод,Пробег,Руль,Состояние,Статус,Таможня,Цвет
819,2016-03-23,Toyota Camry VII (XV50) Рестайлинг в Москве,1033486985,2016-04-05,3520000,Частное лицо (камри),Москва,https://auto.ru/cars/used/sale/1033486985-128b16/,NaN,29661,"3й класс защиты (автомат Калашникова), электро...","[Обивка салона: кожа, Цвет салона: темный, Тон...",NaN,NaN,4 месяца,До января 2018,2016,3.5 л249 л.с.бензин,Автоматическая,Седан,Не интересует,Оригинал,Передний,49 км,Левый,Не требует ремонта,NaN,Растаможен,Белый металлик
